In [1]:
import os,glob
import numpy as np
import pandas as pd
import glob
import pyslha
import time
import progressbar as P
from atlas_susy_2018_13_Recast import getLLPCandidates, getDisplacedJets, getJets
import matplotlib.pyplot as plt
import os,glob
import numpy as np
import seaborn as sns
from scipy.special import kn,zetac
import pyslha
sns.set() #Set style
sns.set_style('ticks',{'font.family':'serif', 'font.serif':'Times New Roman'})
sns.set_context('paper', font_scale=1.8)
sns.set_palette(sns.color_palette("Paired"))

delphesDir = os.path.abspath("./DelphesLLP")
os.environ['ROOT_INCLUDE_PATH'] = os.path.join(delphesDir,"external")

import ROOT

ROOT.gSystem.Load(os.path.join(delphesDir,"libDelphes.so"))

ROOT.gInterpreter.Declare('#include "classes/SortableObject.h"')
ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')


Welcome to JupyROOT 6.30/02


True

In [2]:
inputFile = './pp2GoGo/Events/run_07/gluino_tau_1_delphes_events.root'

In [3]:
f = ROOT.TFile(inputFile,'read')
tree = f.Get("Delphes")
nevts = tree.GetEntries()

wHighPT = 0.0
wTrackless = 0.0
for ievt in range(nevts):    

    tree.GetEntry(ievt)
    llpCandidates = getLLPCandidates(tree.llps,tree.llpDaughters)
    jets = getJets(tree.GenJet,pTmin=50.,etaMax=5.0)
    jetsDisp = getDisplacedJets(tree.GenJet,tree.llps,tree.llpDaughters)
    # Remove displaced jets with decays outside the calorimeter:
    jetsDispFilter = []
    for j in jetsDisp:
        llp = j.llp
        R = np.sqrt(llp.Xd**2 + llp.Yd**2 + llp.Zd**2)
        if R > 3870:
            continue
        jetsDispFilter.append(j)
    
    # Event acceptance:
    highPT_acc = acceptance(jets,jetsDispFilter,sr='HighPT')
    trackless_acc = acceptance(jets,jetsDispFilter,sr='Trackless')                        
    if (not highPT_acc) and (not trackless_acc):
        continue
    
    # Event efficiency
    highPT_eff = eventEff(jets,llpCandidates,sr='HighPT')
    trackless_eff = eventEff(jets,llpCandidates,sr='Trackless')
    
    # Vertex acceptances:
    v_acc = np.array([vertexAcc(llp) for llp in llpCandidates])
    
    # Vertex efficiencies:
    v_eff = np.array([vertexEff(llp) for llp in llpCandidates])
    
    wvertex = 1.0-np.prod(1.0-v_acc*v_eff)
    
    # Add to the total weight in each SR:
    wHighPT += weight*highPT_acc*highPT_eff*wvertex
    wTrackless += weight*highPT_acc*highPT_eff*wvertex
    
    
return {'highPT (pb)' : wHighPT, 'trackless (pb)' : wTrackless}
    


In [ ]:
 def acceptance(jets,jetsDispFilter,sr):
    passAcc = 0.0
    if sr == 'HighPT':
        # Apply HighPT jet selection    
        njet250 = len([j for j in jets if jet.PT > 250.0])
        njet195 = len([j for j in jets if jet.PT > 195.0])
        njet116 = len([j for j in jets if jet.PT > 116.0])
        njet90 = len([j for j in jets if jet.PT > 90.0])
        if (njet250 >= 4) or (njet195 >= 5) or (njet116 >= 6) or (njet90 >= 7):
            passAcc = 1.0
    elif sr == 'Trackless':    
        # Apply Trackless jet selection (only if HighPT has failed)    
        njet137 = len([j for j in jets if jet.PT > 137.0])
        njet101 = len([j for j in jets if jet.PT > 101.0])
        njet83 = len([j for j in jets if jet.PT > 83.0])
        njet55 = len([j for j in jets if jet.PT > 55.0])
        njetDisp70 = len([j for j in jetsDispFilter if jet.PT > 70.0])
        njetDisp50 = len([j for j in jetsDispFilter if jet.PT > 50.0])
        if (njet137 >= 4) or (njet101 >= 5) or (njet83 >= 6) or (njet55 >= 7):
            if (njetDisp70 >=1) or (njetDisp50 >= 2):
                passAcc = 1.0
    
    return passAcc

def vertexAcc(llp,daughters):
    
    passAcc = 1.0
    
    if llp.r_decay > 300.0 or abs(llp.z_decay) > 300.0:
        passAcc = 0.0
        
    if llp.r_decay < 4.0:
        passAcc = 0.0
    
    maxD0 = 0.0
    for d in daughters:
        if d.Charge == 0: # Skip neutral
            continue
        if d.Status != 1: # Skip decayed
            continue
        d0 = abs((d.Y*d.Px - d.X*d.Px)/d.PT)
        maxD0 = max(maxD0,d0)
    if maxD0 < 2.0:
        passAcc = 0.0
        
        
    selectedDP = []
    for d in daughters:
        if d.Charge == 0:
            continue
        if d.Statue != 1:
            continue
        pTratio = abs(d.PT/d.Charge)
        if pTratio < 1.0:
            continue
        selectedDP.append(d)
            
    if len(selectedDP) < 5:
        passAcc = 0.0
        
    pTot = np.zeros(4)
    for d in selectedDP:
        p = np.array([0.,d.Px,d.Py,d.Pz])
        mpion = 0.140
        p[0] = np.sqrt(mpion**2 + np.dot(p[1:],p[1:]))
        pTot += p
    mDV = np.sqrt(pTot[0]**2 - np.dot(pTot[1:],pTot[1:]))
    if mDV < 10.:
        passAcc = 0.0
        
    return passAcc
    

def eventEff(jets,llpCandidates,sr):
    
    jetPT = 0.0
    for j in jets:
        jetPT += j.PT
    r_max = max([llp.r_decay for llp in llpCandidates])
    
    eff = None
    if sr == 'HighPT':
        eff = 
    elif sr == 'Trackless':
        eff = 
        
    return eff

def vertexEff(mDV,nTracks,r_decay):
    
    return eff